## Imports

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
import os
import sys
from os.path import exists
from scipy import spatial

## Settings

In [18]:
im_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced/eval/no_obstacle'
imnames = os.listdir(im_dir)
max_threshold = 0.29
search_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced/eval/no_obstacle'
diffs_base_path = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced/diffs'
diffs_saved_path = '_'.join(search_dir.split('/')[4:])
diffs_path = os.path.join(diffs_base_path, diffs_saved_path)
Path(diffs_path).mkdir(parents=True, exist_ok=True)

## Load the MobileNet Module

In [19]:
module_handle = 'https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4'
module = hub.load(module_handle)

## compare image's ref, current parts

In [20]:
def compare_image(im_dir, im_name):
    im = tf.io.read_file(os.path.join(im_dir, im_name))
    im = tf.io.decode_jpeg(im, channels=3)
    im_src = im 
    w = im.shape[1]
    im1 = im[:, :w//2, :]
    im2 = im[:, w//2:, :]
    src_im1 = im1
    src_im2 = im2
    im1 = tf.image.resize_with_pad(im1, 224, 224)
    im1  = tf.image.convert_image_dtype(im1, tf.float32)[tf.newaxis, ...]
    im2 = tf.image.resize_with_pad(im2, 224, 224)
    im2  = tf.image.convert_image_dtype(im2, tf.float32)[tf.newaxis, ...]  
    f1 = module(im1)
    f1_set = np.squeeze(f1)
    f2 = module(im2)   
    f2_set = np.squeeze(f2)
    diff = spatial.distance.cosine(f1_set, f2_set)
    
    return diff

In [21]:
diff_count = 0

for im_name in tqdm(imnames):
    diff = compare_image(im_dir, im_name)
    if diff >= max_threshold:
        diff_count += 1
        saved_dir = str(max_threshold)
        saved_path = os.path.join(diffs_path, saved_dir)
        Path(saved_path).mkdir(parents=True, exist_ok=True)
        impath = os.path.join(search_dir, im_name)
        im = cv2.imread(impath)
        prefix = '_'.join(im_name.split('_')[:2])
        out_imname = f'{prefix}_{diff:.3f}.jpg'
        cv2.imwrite(os.path.join(saved_path, out_imname), im)

100%|██████████| 1041/1041 [01:32<00:00, 11.23it/s]


In [22]:
im_name

'128_146142_.75_2022-02-01--05-05-46_.jpg'

In [23]:
saved_path

'/home/drevital/obstacles_classification_datasets/rgb_6_balanced/diffs/rgb_6_balanced_eval_no_obstacle/0.29'

In [24]:
diff_count

35